In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from google.colab import drive

In [ ]:
# 1. Mount Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2. Extract RAR file
!apt-get install unrar
!unrar x "/content/drive/MyDrive/Multimodal_SA/1-2000.rar" "/content/memes/"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Multimodal_SA/1-2000.rar

Creating    /content/memes                                            OK
Creating    /content/memes/1-2000                                     OK
Extracting  /content/memes/1-2000/1.jpeg                                   0%  OK 
Extracting  /content/memes/1-2000/10.jpg                                   0%  OK 
Extracting  /content/memes/1-2000/100.jpg                                  0%  OK 
Extracting  /content/memes/1-2000/1000.jpg                                 0%  OK 
Extracting  /content/memes/1-2000/1001.jpg                                 0%  OK 
Extracting  /content/memes/1-2000/1002.jpg 

In [ ]:
# 3. Load Excel File
df = pd.read_excel("/content/drive/MyDrive/Multimodal_SA/Memes.xlsx")

In [ ]:
# 4. Preprocess Labels
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])  # Convert to numerical labels
num_classes = len(label_encoder.classes_)
labels = to_categorical(df['Label'], num_classes=num_classes)

In [ ]:
# 5. Load and Process Images (CNN Feature Extraction)
image_folder = "/content/memes/1-2000"
image_size = (224, 224)

In [ ]:
def load_image(image_id):
    """Loads an image, checking both .jpg and .jpeg extensions."""
    image_id = str(image_id)  # Convert to string
    for ext in [".jpg", ".jpeg"]:
        img_path = os.path.join(image_folder, image_id + ext)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            img = img / 255.0  # Normalize
            return img
    return None  # Return None if the image is missing

In [ ]:
# Load images
images = []
valid_labels = []
valid_texts = []
missing_images = []

for img_id, text, label in zip(df['Meme_ID'], df['Meme_Text'], labels):
    img = load_image(img_id)
    if img is not None:
        images.append(img)
        valid_labels.append(label)  # Include labels only for valid images
        valid_texts.append(text)  # Include texts only for valid images
    else:
        missing_images.append(img_id)

In [ ]:
images = np.array(images)
valid_labels = np.array(valid_labels)

In [ ]:
# Print missing images (if any)
if missing_images:
    print(f"Warning: {len(missing_images)} images not found:", missing_images)

In [ ]:
# 6. Define CNN Model for Image Features
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)
image_model = Model(inputs=base_model.input, outputs=x)
image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# 6. Define CNN Model for Image Features
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
image_output = Dense(num_classes, activation='softmax')(x)
image_model = Model(inputs=base_model.input, outputs=image_output)
image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# 7. Process Text Data (Tokenization and Padding)
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(valid_texts)
sequences = tokenizer.texts_to_sequences(valid_texts)
padded_texts = pad_sequences(sequences, maxlen=max_len)

In [ ]:
len(padded_texts), len(valid_labels)

(1749, 1749)

In [ ]:
# 8. Define Text Model
text_input = Input(shape=(max_len,))
x = Dense(128, activation='relu')(text_input)
text_output = Dense(num_classes, activation='softmax')(x)
text_model = Model(inputs=text_input, outputs=text_output)
text_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 9. Train Separate Models
X_train_img, X_test_img, y_train, y_test = train_test_split(images, valid_labels, test_size=0.2, random_state=42)
X_train_text, X_test_text, _, _ = train_test_split(padded_texts, valid_labels, test_size=0.2, random_state=42)

In [ ]:
image_model.fit(X_train_img, y_train, epochs=10, batch_size=32, validation_data=(X_test_img, y_test))
text_model.fit(X_train_text, y_train, epochs=10, batch_size=32, validation_data=(X_test_text, y_test))

Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.2954 - loss: 2.7600 - val_accuracy: 0.3914 - val_loss: 1.9407
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 36s 146ms/step - accuracy: 0.6243 - loss: 0.9911 - val_accuracy: 0.3600 - val_loss: 2.8910
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - accuracy: 0.8531 - loss: 0.4270 - val_accuracy: 0.2057 - val_loss: 4.4446
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 146ms/step - accuracy: 0.9396 - loss: 0.2364 - val_accuracy: 0.2086 - val_loss: 3.4324
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 143ms/step - accuracy: 0.9640 - loss: 0.1375 - val_accuracy: 0.2143 - val_loss: 2.1747
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 149ms/step - accuracy: 0.9700 - loss: 0.1192 - val_accuracy: 0.3314 - val_loss: 3.2289
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 149ms/step - accuracy: 0.9750 - loss: 0.0911 - val_accuracy: 0.2314 - val_loss: 4.6527
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 148ms/step - accuracy: 0.9758 - loss: 0.0812 - val_accura

In [ ]:
# 10. Perform Late Fusion (Combine Predictions)
image_preds = image_model.predict(X_test_img)
text_preds = text_model.predict(X_test_text)

11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 591ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [ ]:
# Weighted average of predictions (equal weight fusion)
final_preds = (image_preds + text_preds) / 2
final_labels = np.argmax(final_preds, axis=1)
true_labels = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report

# Get unique labels present in y_test
unique_labels = np.unique(true_labels)

# Generate classification report with only existing labels
print(classification_report(true_labels, final_labels, labels=unique_labels, target_names=[label_encoder.classes_[i] for i in unique_labels]))

              precision    recall  f1-score   support

    Negative       0.22      0.21      0.21        73
   Negative        0.12      0.10      0.11        40
     Neutral       0.43      0.44      0.44       137
    Neutral        0.18      0.18      0.18        60
    Positive       0.14      0.18      0.16        33
   Positive        0.00      0.00      0.00         6
     neutral       0.00      0.00      0.00         1

   micro avg       0.28      0.27      0.27       350
   macro avg       0.16      0.16      0.16       350
weighted avg       0.27      0.27      0.27       350

